# Fast.ai RESNET-50

In [36]:
# standard modules
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# pytorch modules
import torch as torch
from torch import nn
from torch import conv2d, convolution
import torchvision.models as models
from torchvision.models import * 
# fast.ai modules
from fastai import *
from fastai.basics import *
from fastai.script import *
from fastai.vision import *
from fastai.vision.models.wrn import wrn_22
from fastai.distributed import *
# GPU config
torch.backends.cudnn.benchmark = True

In [24]:
path = Path('data/')

torch.device('cuda', 0)
#data = ImageDataBunch.from_folder(path, size=224, valid_pct=20)

device(type='cuda', index=0)

In [25]:
for folder in ('NORMAL', 'CNV', 'DME', 'DRUSEN'):
    print(folder)
  #  verify_images(path/folder, delete=True, max_size=500)

NORMAL
CNV
DME
DRUSEN


In [27]:
data.classes

['CNV', 'DME', 'DRUSEN', 'NORMAL', 'test']

In [31]:
os.listdir('data/validation')

['.DS_Store', 'CNV', 'DME', 'DRUSEN', 'NORMAL']

In [ ]:
data.path = pathlib.Path('.')

# build train model 5 epoch
learn = create_cnn(data,
                   resnet50,
                   pretrained = True,
                   metrics = accuracy)
learn.fit_one_cycle(5)

# unfreeze and update learning rate
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()
learn.fit_one_cycle(3,max_lr = slice(1e-6,3e-4))

# interpretation, cm, loss
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9)
interp.plot_confusion_matrix()
preds,y = learn.TTA()
acc = accuracy(preds, y)
print('The validation accuracy is {} %.'.format(acc * 100))

def generateSubmission(learner):
    submissions = pd.read_csv('../input/sample_submission.csv')
    id_list = list(submissions.id)
    preds,y = learner.TTA(ds_type=DatasetType.Test)
    pred_list = list(preds[:,1])
    pred_dict = dict((key, value.item()) for (key, value) in zip(learner.data.test_ds.items,pred_list))
    pred_ordered = [pred_dict[Path('../input/test/' + id + '.tif')] for id in id_list]
    submissions = pd.DataFrame({'id':id_list,'label':pred_ordered})
    submissions.to_csv("submission_{}.csv".format(pred_score),index = False)

generateSubmission(learn)

C:\Users\Blair\Anaconda3\envs\oct\lib\site-packages\fastai\vision\learner.py:106: UserWarning: `create_cnn` is deprecated and is now named `cnn_learner`.
  warn("`create_cnn` is deprecated and is now named `cnn_learner`.")
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\Blair/.cache\torch\checkpoints\resnet50-19c8e357.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [00:04<00:00, 21.2MB/s]


epoch,train_loss,valid_loss,accuracy,time
0,0.293990,0.214711,0.928741,16:32
